In [1]:

import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
yf.pdr_override()
from dateutil import parser

In [2]:
def build_fcff(tick,cost_of_capital,rev_growth,time_periods,target_pretax_operatingmargin,terminal_growth_rate):
    '''
    cost_of_capital
    rev_growth
    time_periods
    target_pretax_operatingmargin
    terminal_growth_rate
    '''
    ticker=yf.Ticker(tick)
    
    income_statement_df=ticker.financials
    income_statement_df=income_statement_df.astype('float64')
    info=ticker.get_info()
    
    info=pd.DataFrame.from_dict(info,orient='index')
    
    bal_sheets=ticker.get_balance_sheet()
    bal_sheets=bal_sheets.iloc[:,0:1]
    dcf=pd.DataFrame(columns=income_statement_df.columns,index=['revenue','operating_income',
                                                                'ebit_operating_margin','income_tax_rate','income_before_tax',
                                                               'ebit_1-t','reinvestment%','reinvestment','fcff',
                                                                'cost_of_capital%','cumulative_disc_factor'])
    time_periods_col=time_periods+2

        
    proj_dcf=pd.DataFrame(columns=range(time_periods_col),index=dcf.index.copy())
    
    for i in income_statement_df:
        
        revenue= income_statement_df.loc['Total Revenue'][i]
        dcf.loc['revenue'][i]=revenue
        
           
        OperatingIncome=income_statement_df.loc['Operating Income'][i]
        dcf.loc['operating_income'][i]=OperatingIncome
        
        ebit_operating_margin=OperatingIncome/revenue
        dcf.loc['ebit_operating_margin'][i]=ebit_operating_margin
        
        Incometax=income_statement_df.loc['Income Tax Expense'][i]
        dcf.loc['income_tax_rate'][i]=abs(Incometax/OperatingIncome)
        
        IncomeBeforeTax=income_statement_df.loc['Income Before Tax'][i]
        dcf.loc['income_before_tax'][i]=IncomeBeforeTax
        
        ebit=IncomeBeforeTax*(1-(Incometax/OperatingIncome))
        dcf.loc['ebit_1-t'][i]=ebit
        
        reinvestment= dcf.loc['revenue'].pct_change(1).shift(-1)
        dcf.loc['reinvestment%']=1-reinvestment
        
        reinvest_cash=(reinvestment*revenue)/1
        dcf.loc['reinvestment']=round(reinvest_cash,2)
        
        fcff=ebit-reinvest_cash
        dcf.loc['fcff']=round(fcff,2)
        
        dcf.loc['cost_of_capital%']=cost_of_capital
        
        dcf.loc['cumulative_disc_factor']=100/(100+cost_of_capital)
        
    
    proj_dcf.loc['fcffPV']=0
    proj_dcf.iloc[:,0]=dcf.iloc[:,0]
    proj_dcf.loc['rev_growth']=rev_growth
    linear=np.linspace(rev_growth,0,num=6)
   
    t=0
    try:
            
        for k in range(time_periods_col):
                
                proj_dcf.loc['revenue'][k+1]=round((1+(rev_growth/100))*proj_dcf.loc['revenue'][k],2)
                proj_dcf.loc['operating_income'][k+1]=round((1+(target_pretax_operatingmargin/100))*proj_dcf.loc['operating_income'][k],2)
                proj_dcf.loc['ebit_operating_margin']=1-(proj_dcf.loc['operating_income']/proj_dcf.loc['revenue'])
                proj_dcf.loc['income_tax_rate'][k+1]=proj_dcf.loc['income_tax_rate'][k]
                proj_dcf.loc['income_before_tax'][k]=proj_dcf.loc['operating_income'][k]*proj_dcf.loc['ebit_operating_margin'][k]                
                proj_dcf.loc['ebit_1-t'][k]=round(((1+proj_dcf.loc['income_tax_rate'][k]*0.1))*proj_dcf.loc['operating_income'][k],2)
                proj_dcf.loc['reinvestment'][k]=(proj_dcf.loc['revenue'][k+1]- proj_dcf.loc['revenue'][k])/1
                proj_dcf.loc['fcff'][k]=proj_dcf.loc['income_before_tax'][k]-proj_dcf.loc['reinvestment'][k]
                proj_dcf.loc['cost_of_capital%'][k]=cost_of_capital
                proj_dcf.loc['cumulative_disc_factor'][k+1]=proj_dcf.loc['cumulative_disc_factor'][k]*(1/(1+(proj_dcf.loc['cost_of_capital%'][k]*.01)))
                proj_dcf.loc['fcffPV'][k]=round(proj_dcf.loc['fcff'][k]*proj_dcf.loc['cumulative_disc_factor'][k],2)
                if k>5:
                    proj_dcf.loc['rev_growth'][k]=linear[t]

                    t=t+1
                    proj_dcf.loc['revenue'][k+1]=round((1+(rev_growth/100))*proj_dcf.loc['revenue'][k],2)
                    proj_dcf.loc['operating_income'][k+1]=round((1+(target_pretax_operatingmargin/100))*proj_dcf.loc['operating_income'][k],2)
                    proj_dcf.loc['ebit_operating_margin']=1-(proj_dcf.loc['operating_income']/proj_dcf.loc['revenue'])
                    proj_dcf.loc['income_tax_rate'][k+1]=proj_dcf.loc['income_tax_rate'][k]
                    proj_dcf.loc['income_before_tax'][k]=proj_dcf.loc['operating_income'][k]*proj_dcf.loc['ebit_operating_margin'][k]
                  
                    proj_dcf.loc['ebit_1-t'][k]=round(((1+proj_dcf.loc['income_tax_rate'][k]*0.1))*proj_dcf.loc['operating_income'][k],2)
                    proj_dcf.loc['reinvestment'][k]=(proj_dcf.loc['revenue'][k+1]- proj_dcf.loc['revenue'][k])/1
                    proj_dcf.loc['fcff'][k]=proj_dcf.loc['income_before_tax'][k]-proj_dcf.loc['reinvestment'][k]
                    proj_dcf.loc['cost_of_capital%'][k]=cost_of_capital
                    proj_dcf.loc['cumulative_disc_factor'][k+1]=proj_dcf.loc['cumulative_disc_factor'][k]*(1/(1+(proj_dcf.loc['cost_of_capital%'][k]*.01)))

    except KeyError:
                
        pass

            
    proj_dcf=proj_dcf.iloc[:,:-1]
    proj_dcf=proj_dcf.astype('float64')
    term_yr=[]
    term_yr=proj_dcf.iloc[:,10]  
    
    output_list=['Terminal cash flow','terminal value','terminal cost of capital','PV Terminal value','PV of CF over timeperiod',
                'sum of pv','Value of operating assets','debt','cash','value_of_EQ','value_of_EQ_to_common','#_of_shs',
                'estimated value']
    output=pd.DataFrame(index=output_list,columns=['value'],data=term_yr)
    output.loc['terminal cost of capital']=term_yr['cost_of_capital%']    
    output.loc['Terminal cash flow']=round(term_yr.loc['fcff'],2)-term_yr['reinvestment']
    output.loc['terminal value']=output.loc['Terminal cash flow']/(1+(term_yr['cost_of_capital%']/100))
    output.loc['PV Terminal value']=output.loc['terminal value']*term_yr['cumulative_disc_factor']        
    output.loc['cash']=bal_sheets.loc['Cash'][0]
    output.loc['debt']=bal_sheets.loc['Long Term Debt'][0]    
    output.loc['PV of CF over timeperiod']=proj_dcf.loc['fcffPV'].sum()
    output.loc['sum of pv']=output.loc['PV of CF over timeperiod']
    output.loc['Value of operating assets']=output.loc['sum of pv']       
    output.loc['value_of_EQ']=output.loc['Value of operating assets']-output.loc['debt']+output.loc['cash']
    output.loc['value_of_EQ_to_common']=output.loc['value_of_EQ'] #-output.loc['debt']
    output.loc['#_of_shs']=info.loc['sharesOutstanding'][0]     
    output.loc['estimated value']=(output.loc['value_of_EQ']/output.loc['#_of_shs']  )
    
    
        
                                                      
    return dcf,proj_dcf,output,term_yr
 

In [3]:


dcf,proj_dcf,output,term_yr=build_fcff('ebc',cost_of_capital=5.0,rev_growth=7,time_periods=10,
                        terminal_growth_rate=5,target_pretax_operatingmargin=7)

In [4]:
dcf

,2020-12-31,2019-12-31,2018-12-31,2010-12-31
revenue,530045000.0,587263000.0,555539000.0,351073000.0
operating_income,137854000.0,192904000.0,178933000.0,97803000.0
ebit_operating_margin,0.26008,0.32848,0.322089,0.278583
income_tax_rate,0.095485,0.204667,0.194956,0.246301
income_before_tax,35901000.0,174579000.0,157611000.0,84527000.0
ebit_1-t,32472990.199777,138848514.893418,126883844.450157,63707895.238387
reinvestment%,0.892051,1.05402,1.36805,NaN
reinvestment,37898093.4,-18964994.99,-129212336.16,NaN
fcff,25809801.84,82672890.23,192920231.4,NaN
cost_of_capital%,5.0,5.0,5.0,5.0


In [5]:
proj_dcf

,0,1,2,3,4,5,6,7,8,9,10
revenue,5.300450e+08,5.671482e+08,6.068485e+08,6.493279e+08,6.947809e+08,7.434155e+08,7.954546e+08,8.511364e+08,9.107160e+08,9.744661e+08,1.042679e+09
operating_income,1.378540e+08,1.475038e+08,1.578290e+08,1.688771e+08,1.806985e+08,1.933474e+08,2.068817e+08,2.213634e+08,2.368588e+08,2.534390e+08,2.711797e+08
ebit_operating_margin,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01,7.399202e-01
income_tax_rate,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02,9.548508e-02
income_before_tax,1.020010e+08,1.091410e+08,1.167809e+08,1.249556e+08,1.337024e+08,1.430616e+08,1.530759e+08,1.637912e+08,1.752566e+08,1.875246e+08,2.006513e+08
ebit_1-t,1.391703e+08,1.489122e+08,1.593361e+08,1.704896e+08,1.824239e+08,1.951935e+08,2.088571e+08,2.234771e+08,2.391205e+08,2.558589e+08,2.737690e+08
reinvestment%,8.920507e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reinvestment,3.710315e+07,3.970037e+07,4.247940e+07,4.545295e+07,4.863466e+07,5.203909e+07,5.568182e+07,5.957955e+07,6.375012e+07,6.821263e+07,7.298751e+07
fcff,6.489781e+07,6.944066e+07,7.430150e+07,7.950261e+07,8.506779e+07,9.102253e+07,9.739411e+07,1.042117e+08,1.115065e+08,1.193120e+08,1.276638e+08
cost_of_capital%,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [6]:
output

,value
Terminal cash flow,54676300.44
terminal value,52072667.085714
terminal cost of capital,5.0
PV Terminal value,30445809.97251
PV of CF over timeperiod,748477274.84
sum of pv,748477274.84
Value of operating assets,748477274.84
debt,28049000.0
cash,2004870000.0
value_of_EQ,2725298274.84


In [7]:
term_yr

revenue                   1.042679e+09
operating_income          2.711797e+08
ebit_operating_margin     7.399202e-01
income_tax_rate           9.548508e-02
income_before_tax         2.006513e+08
ebit_1-t                  2.737690e+08
reinvestment%                      NaN
reinvestment              7.298751e+07
fcff                      1.276638e+08
cost_of_capital%          5.000000e+00
cumulative_disc_factor    5.846793e-01
fcffPV                    7.464239e+07
rev_growth                1.400000e+00
Name: 10, dtype: float64

In [8]:
def ggm_stable_growth(tick,cost_of_eq,exp_growth_rate):
    ticker=yf.Ticker(tick)
    bal_sheets=ticker.get_balance_sheet()
    index_list=['tick','currentdiv','payoutRatio','exp_growth_rate','trailingEps','earningsQuarterlyGrowth',
                'cost_of_eq','ggm_calc','mkt_price']
    ggm_df=pd.DataFrame(index=index_list,columns=['Value'])
    
    income_statement=ticker.financials
    
    cashflow=ticker.cashflow
    info=ticker.get_info()
    info=pd.DataFrame.from_dict(info,orient='index') 
    eps=info.loc['trailingEps'][0]
    beta=info.loc['beta'][0]
    regularMarketPrice=info.loc['regularMarketPrice'][0]
    payoutRatio=info.loc['payoutRatio'][0]
    earn_growth=info.loc['earningsQuarterlyGrowth'][0]
        
    current_div=eps*payoutRatio

    cost_of_eq=(cost_of_eq/100)
    exp_growth_rate=exp_growth_rate/100
    exp_growth_rate_num=1+exp_growth_rate

    ggm=(current_div*(exp_growth_rate_num))/(cost_of_eq-exp_growth_rate)

    ggm_df.loc['tick']=tick
    ggm_df.loc['currentdiv']=round(current_div,2)
    ggm_df.loc['payoutRatio']=round(payoutRatio,2)
    ggm_df.loc['exp_growth_rate']=exp_growth_rate
    ggm_df.loc['earningsQuarterlyGrowth']=earn_growth
    ggm_df.loc['trailingEps']=eps
    ggm_df.loc['cost_of_eq']=cost_of_eq
    ggm_df.loc['ggm_calc']=round(ggm,2)
    ggm_df.loc['mkt_price']=regularMarketPrice
    return ggm_df
    

In [9]:
ggm_stable_growth('ebc',cost_of_eq=6,exp_growth_rate=5)

,Value
tick,ebc
currentdiv,0.23
payoutRatio,0.54
exp_growth_rate,0.05
trailingEps,0.431
earningsQuarterlyGrowth,0.302
cost_of_eq,0.06
ggm_calc,24.62
mkt_price,21.62
